In [6]:
import pandas as pd
import bar_chart_race as bcr
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


csv_file_path = r'music_최종.CSV'
df = pd.read_csv(csv_file_path, encoding='cp949')


df['Month'] = pd.to_datetime(df['Month'], format='%Y%m')

# 같은 월에 같은 가수의 노래 중 첫 번째 노래만을 카운트
df['상위_50위_누적_횟수'] = df[df['순위'] <= 50].sort_values(['가수', 'Month']).drop_duplicates(['가수', 'Month']).groupby('가수').cumcount() + 1

# 월마다 기록된  순위 50위에 들어간 횟수를 기록하는 새로운 열 추가
df['top50_카운트'] = df[df['순위'] <= 50].sort_values(['가수', 'Month']).drop_duplicates(['가수', 'Month']).groupby('가수')['상위_50위_누적_횟수'].apply(lambda x: x.shift().fillna(0) + 1)


# 가수 이름을 열로, 인덱스를 Month로 설정하여 피벗
pivot_df = df.pivot_table(values='top50_카운트', index='Month', columns='가수', aggfunc='sum')


#결측값 채우기
pivot_df.fillna(method='ffill', inplace=True)
pivot_df.fillna(0, inplace=True)

# 폰트 지정
font_path = '/Users/yoosukyeong/Library/Fonts/LeferiPointBlack.ttf'  
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name)

bcr.bar_chart_race(pivot_df, period_length=500, n_bars=10, sort='desc',
                     fixed_order=False,fixed_max=True,
                   orientation='h',
                   dpi=144,
                   figsize=(12, 6), period_fmt='%Y년-%m월',
                   title='TOP 50 진입 횟수 ', filename = 'top 50 진입 횟수.mp4')

# 참고 https://www.dexplo.org/bar_chart_race/data_preparation/
# https://www.dunderdata.com/blog/create-a-bar-chart-race-animation-in-python-with-matplotlib

TypeError: incompatible index of inserted column with frame index